In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

from seaborn import lmplot, stripplot, scatterplot, heatmap

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_val_score

from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix

<h1>Read in and Check the Data</h1>

In [3]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("../datasets/dataset_salaries.csv")

# Shuffle the dataset
df = df.sample(frac=1, random_state=2)
df.reset_index(drop=True, inplace=True)

In [4]:
df.shape

(62642, 17)

In [5]:
df.columns

Index(['timestamp', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber'],
      dtype='object')

<p>
    Inspect the data and determine whether there is any cleaning needed.
</p>

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62527 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  int64  
 7   yearsatcompany           62642 non-null  int64  
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  int64  
 10  stockgrantvalue          62642 non-null  int64  
 11  bonus                    62642 non-null  int64  
 12  gender                   43102 non-null  object 
 13  otherdetails             40139 non-null  object 
 14  cityid                

In [7]:
df.describe(include='all')

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber
count,62642,62637,62527,62642,6.264200e+04,62642,62642.000000,62642.000000,61788,6.264200e+04,6.264200e+04,62642.000000,43102,40139,62642.000000,62640.000000,62642.000000
unique,62561,1869,3071,15,NaN,1050,NaN,NaN,3271,NaN,NaN,NaN,4,40132,NaN,NaN,NaN
top,1/10/2019 21:44:02,Amazon,L4,Software Engineer,NaN,"Seattle, WA",NaN,NaN,Full Stack,NaN,NaN,NaN,Male,twauesly,NaN,NaN,NaN
freq,3,8054,5008,41231,NaN,8701,NaN,NaN,11382,NaN,NaN,NaN,35702,2,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,3.175327e+03,NaN,7.191054,2.700616,NaN,1.798145e+03,1.134865e+03,242.317199,NaN,NaN,9856.201989,616.097222,41694.723732
std,NaN,NaN,NaN,NaN,3.877877e+04,NaN,6.437148,3.524030,NaN,1.927194e+04,2.520611e+04,3709.187697,NaN,NaN,6679.104563,289.843390,24488.865879
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,0.000000,0.000000,NaN,0.000000e+00,0.000000e+00,0.000000,NaN,NaN,0.000000,0.000000,1.000000
25%,NaN,NaN,NaN,NaN,1.160000e+02,NaN,3.000000,0.000000,NaN,8.800000e+01,0.000000e+00,1.000000,NaN,NaN,7369.000000,506.000000,20069.250000
50%,NaN,NaN,NaN,NaN,1.830000e+02,NaN,6.000000,1.000000,NaN,1.310000e+02,2.300000e+01,13.000000,NaN,NaN,7839.000000,807.000000,42019.000000
75%,NaN,NaN,NaN,NaN,2.690000e+02,NaN,10.000000,4.000000,NaN,1.810000e+02,6.400000e+01,27.000000,NaN,NaN,11521.000000,807.000000,63021.750000


In [8]:
# explain why removed some attributes
features = ['timestamp', 'company', 'level', 'title', 'location', 'yearsofexperience', 'yearsatcompany', 'tag',
       'stockgrantvalue', 'bonus', 'gender']

In [9]:
nominal_features = ['company', 'level', 'title', 'location', 'tag', 'gender']

In [10]:
numeric_features = ['yearsofexperience', 'yearsatcompany', 'stockgrantvalue', 'bonus']

In [11]:
# Extract the features but leave as a DataFrame
X = df[features]

# Target values, converted to a 1D numpy array
y = df['basesalary']

In [12]:
resulting_df = pd.concat([X, y])

In [13]:
df = df.filter(resulting_df)

In [14]:
df.replace("", np.nan)

,timestamp,company,level,title,location,yearsofexperience,yearsatcompany,tag,stockgrantvalue,bonus,gender
0,11/3/2020 10:26:30,Microsoft,60,Data Scientist,"Redmond, WA",1,1,ML / AI,47,30,Male
1,8/14/2018 21:08:29,VMware,Senior MTS,Software Engineer,"Palo Alto, CA",5,3,Full Stack,0,0,NaN
2,5/18/2019 0:32:17,Paypal,T25,Software Engineer,"San Francisco, CA",7,1,API Development (Back-End),19,14,Male
3,8/3/2021 20:25:07,Trend Micro,Senior Engineer,Software Engineer,"Taipei, TP, Taiwan",2,2,Distributed Systems (Back-End),1,3,Male
4,5/3/2021 15:25:42,Visa,Senior Software Engineer,Software Engineer,"Austin, TX",3,0,Full Stack,9,12,Female
...,...,...,...,...,...,...,...,...,...,...,...
62637,3/3/2021 14:14:45,Nvidia,IC4,Hardware Engineer,"Santa Clara, CA",9,3,Verification,145,0,Male
62638,9/10/2020 8:53:01,Qualcomm,Principal Engineer,Hardware Engineer,"Austin, TX",15,13,ASIC Design,87,41,Male
62639,4/22/2019 13:18:06,Microsoft,61,Software Engineer,"Redmond, WA",6,1,Full Stack,4,7,Male
62640,11/3/2020 19:56:58,Google,L3,Business Analyst,"Tokyo, TY, Japan",3,1,Technical,12,7,Male


In [15]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [16]:
df = pd.to_numeric(df[numeric_features])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:
for feature in nominal_features:
    print(feature, df[feature].unique())

In [ ]:
# Split off the test set: 20% of the dataset.
dev_df, test_df = train_test_split(df, train_size=0.8, random_state=2)

In [ ]:
dev_X = dev_df[features]
test_X = test_df[features]

In [ ]:
dev_y = dev_df['basesalary'].values
test_y = test_df['basesalary'].values

In [ ]:
ss = ShuffleSplit(n_splits=1, train_size=0.75, random_state=2)
ss

In [ ]:
# It can be good to do this on a copy of the dataset (excluding the test set, of course)
copy_df = dev_df.copy()

In [ ]:
m = scatter_matrix(copy_df, figsize=(15, 15))

In [ ]:
heatmap(copy_df.corr(numeric_only=True), annot=True)

In [ ]:
# Extract the features but leave as a DataFrame
dev_X = dev_df[features]
test_X = test_df[features]

dev_y = dev_df['basesalary']
test_y = test_df['basesalary']

In [ ]:
preprocessor = ColumnTransformer([
        ("num", Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
                        ("scaler", StandardScaler())]), numeric_features),
        ("nom", Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
                        ("binarizer", OneHotEncoder(handle_unknown="ignore"))]), nominal_features)],
        remainder="passthrough")

In [ ]:
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=2)
ss

In [ ]:
def build_model(model, additional_params={}):
    model = Pipeline([
        ("preprocessor", preprocessor),
        ("predictor", model)])

    model_param_grid = {
#         "preprocessor__num__uwr__insert": [True, False],
#         "preprocessor__num__units_elapsed__insert": [True, False],
    }
    
    model_param_grid.update(additional_params)

    model_gs = GridSearchCV(model, model_param_grid, scoring="neg_mean_absolute_error", cv=ss, refit=True)

    model_gs.fit(dev_X, dev_y)

    print(model_gs.best_params_, model_gs.best_score_)
    
    return model_gs

In [ ]:
knn_gs = build_model(KNeighborsRegressor(), additional_params={"predictor__n_neighbors": [x for x in range(1, 3)]})

In [ ]:
# scaling wouldn't do any harm
linear_gs = build_model(LinearRegression())

In [ ]:
cross_val_score(knn_gs, X, y, scoring="neg_mean_absolute_error", cv=10)